In [118]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.vector_ar.var_model import VAR

In [ ]:
# Reading the CSV file.
df = pd.read_csv('weather.csv')

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
#sort value by timestamp
df = df.sort_values(by="TIMESTAMP",ascending=True)
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'],unit='s')
df = df.set_index("TIMESTAMP")

In [ ]:
!pip install sweetviz

In [ ]:
import sweetviz as sv
feature_config = sv.FeatureConfig(force_num=['M_PACKET_FORMAT', 'M_GAME_MAJOR_VERSION', 'M_GAME_MINOR_VERSION',
       'M_PACKET_VERSION', 'M_PACKET_ID', 'M_SESSION_UID', 'M_SESSION_TIME',
       'M_FRAME_IDENTIFIER', 'M_PLAYER_CAR_INDEX',
       'M_SECONDARY_PLAYER_CAR_INDEX', 'M_BRAKING_ASSIST',
       'M_SESSION_LINK_IDENTIFIER', 'M_PIT_RELEASE_ASSIST',
       'M_ZONE_START', 'M_ZONE_FLAG', 'M_PIT_STOP_WINDOW_IDEAL_LAP',
       'M_TRACK_TEMPERATURE', 'M_TRACK_LENGTH', 'M_GAME_PAUSED',
       'M_FORECAST_ACCURACY', 'M_AIR_TEMPERATURE',
       'M_NUM_WEATHER_FORECAST_SAMPLES', 'M_SLI_PRO_NATIVE_SUPPORT',
       'M_SAFETY_CAR_STATUS', 'M_TRACK_ID', 'M_ERSASSIST', 'M_FORMULA',
       'M_SEASON_LINK_IDENTIFIER', 'M_PIT_ASSIST', 'M_GEARBOX_ASSIST',
       'M_SESSION_TYPE', 'M_SPECTATOR_CAR_INDEX',
       'M_PIT_STOP_WINDOW_LATEST_LAP', 'M_WEEKEND_LINK_IDENTIFIER',
       'M_DYNAMIC_RACING_LINE_TYPE', 'M_SESSION_TIME_LEFT',
       'M_SESSION_DURATION', 'M_PIT_STOP_REJOIN_POSITION',
       'M_WEATHER_FORECAST_SAMPLES_M_SESSION_TYPE', 'M_TIME_OFFSET',
       'M_WEATHER_FORECAST_SAMPLES_M_WEATHER',
       'M_WEATHER_FORECAST_SAMPLES_M_TRACK_TEMPERATURE',
       'M_TRACK_TEMPERATURE_CHANGE',
       'M_WEATHER_FORECAST_SAMPLES_M_AIR_TEMPERATURE',
       'M_AIR_TEMPERATURE_CHANGE', 'M_RAIN_PERCENTAGE', 'M_WEATHER',
       'M_AI_DIFFICULTY', 'M_PIT_SPEED_LIMIT', 'M_NETWORK_GAME',
       'M_TOTAL_LAPS', 'M_STEERING_ASSIST', 'M_IS_SPECTATING',
       'M_DYNAMIC_RACING_LINE', 'M_DRSASSIST', 'M_NUM_MARSHAL_ZONES'])




In [ ]:
report = sv.analyze(df, target_feat='M_WEATHER', feat_cfg=feature_config)

In [ ]:
report.show_html()

Use PCA to do dimensionality reduction and identify the significant feactures
-----------------------------------------------------------------------------

**Function to :**

* drop weather column and rain percentage and split data to test set
* split data to test and train set
* Normalize data



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def dropy(data):
    newy1 = data["M_WEATHER"]
    newy2 = data["M_RAIN_PERCENTAGE"]
    newx = data.drop(["M_WEATHER","M_RAIN_PERCENTAGE"],axis=1)
    return newy1,newy2,newx

def split(x,y):
    [x_train,x_test,y_train,y_test] = train_test_split(x,y, test_size=0.33, random_state=1)
    return x_train,x_test,y_train,y_test
    
def norm(data,features):
    x = data.loc[:, features].values
    x = StandardScaler().fit_transform(x) # normalizing the features
    #convert normalized features into dataframe
    x = pd.DataFrame(x,index = data.index ,columns=features)
    return x

def split_2(test,train,x,y):
    rows = x.shape[0]
    test_portion = round(test*rows)
    train_portion = round(train*rows)
    x_train = x[0:train_portion]
    x_test = x[x_train.shape[0]+1:-1]
    y_train = y[0:train_portion]
    y_test = y[y_train.shape[0]+1:-1]
    return x_train, x_test, y_train, y_test
    

In [119]:
weather, rain_percent, subset1 = dropy(df)
predictor_weather = ["M_TRACK_TEMPERATURE","M_AIR_TEMPERATURE"]#"M_TRACK_TEMPERATURE","M_TRACK_ID"]
predictor_weather_val = norm(subset1,predictor_weather) #normalize predictors value
np.std(predictor_weather_val) #prove that predictor values has 0 mean and 1 standard deviation
#split predictor_weather_valinto test and train set
x_train,x_test,y_train,y_test = train_test_split(predictor_weather_val,weather)

In [120]:
predictor_w_weather = ["M_TRACK_TEMPERATURE","M_AIR_TEMPERATURE","M_WEATHER"]
subset2 = norm(df,predictor_w_weather)
x_train_w,x_test_w,y_train_w,y_test_w = split_2(0.3,0.7,subset2,subset2["M_WEATHER"])


In [ ]:
from sklearn.decomposition import PCA
pca_weather = PCA(n_components=2)
pc_weather = pca_weather.fit_transform(x_train)
pc_weather_explained = pca_weather.explained_variance_ratio_
pc_weather_explained

In [ ]:
from sklearn.linear_model import LinearRegression
# instantiate the regressor class
regressor = LinearRegression()

# fit the build the model by fitting the regressor to the training data
regressor.fit(x_train, y_train)

# make a prediction set using the test set
prediction = regressor.predict(x_test)

# Evaluate the prediction accuracy of the model
from sklearn.metrics import mean_absolute_error, median_absolute_error
print("The Explained Variance: %.2f" % regressor.score(x_test, y_test))
print("The Mean Absolute Error: %.2f  " % mean_absolute_error(y_test, prediction))
print("The Median Absolute Error: %.2f  " % median_absolute_error(y_test, prediction))

Weather Predictive Model Building
--------------------------------------


In [121]:
#fit the model

model = VAR(x_train_w)
#model_fit = model.fit()
x = model.select_order(maxlags=12)
x.summary()



/home/datascience/conda/generalmachinelearningforcpusonpython3_7vyes/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


,AIC,BIC,FPE,HQIC
0,-4.573,-4.573,0.01032,-4.573
1,-29.04,-29.04,2.440e-13,-29.04
2,-29.21,-29.21,2.060e-13,-29.21
3,-29.22,-29.22,2.042e-13,-29.22
4,-29.22,-29.22,2.041e-13,-29.22
5,-29.22,-29.22,2.041e-13,-29.22
6,-29.22,-29.22,2.041e-13,-29.22
7,-29.22,-29.22,2.040e-13,-29.22
8,-29.22,-29.22,2.039e-13,-29.22
9,-29.22,-29.22,2.038e-13,-29.22


In [ ]:
model_fitted = model.fit(3)

In [ ]:
model_fitted.summary()


In [ ]:
from statsmodels.stats.stattools import durbin_watson
out = durbin_watson(model_fitted.resid)

 #the closer the residual to 2, the less significant the residual
    
for col, val in zip(x_train_w.columns, out):
    print(col, ':', round(val, 2))
    

In [ ]:
# Forecast Input
x_test_w_lag = x_test_w[0:3] #forecast input
x_test_w_lag_noindex = x_test_w_lag.reset_index(drop=True)
x_train_w_lag = x_train_w[-3:].values

In [ ]:
start_date = x_train_w.index
start_date = list(start_date)[-1]


In [ ]:
start_date = start_date.to_timestamp()

In [ ]:
fc = model_fitted.forecast(x_train_w_lag,steps=5)
idx = pd.date_range(start_date, periods=5, freq='T')
df_forecast=pd.DataFrame(data=fc, index=idx, columns=x_test_w_lag.columns)
df_forecast

Evaluate the Forecast
---------------------

Rain_Percentage 
---------------

In [ ]:
subset_rain = df[df["M_RAIN_PERCENTAGE"].notna()]

Based on the Pearson's Correlation from the analysis above, the below features is chosen
----------------------------------------------------------------------------------------

In [ ]:
predictors_rain = ['M_WEATHER_FORECAST_SAMPLES_M_WEATHER',
       'M_WEATHER_FORECAST_SAMPLES_M_TRACK_TEMPERATURE',
       'M_TRACK_TEMPERATURE_CHANGE',
       'M_WEATHER_FORECAST_SAMPLES_M_AIR_TEMPERATURE',
       'M_AIR_TEMPERATURE_CHANGE',""]

In [117]:
#target variable
target2 = subset_rain["M_RAIN_PERCENTAGE"]
#normalized 
subset_rain = norm(subset_rain,predictors_rain)
#split to test train
x_train_rain,x_test_rain,y_train_rain,y_test_rain = split(subset_rain,target2)

Rain Percentage Predictive Model Building
------------------

In [ ]:
#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR


model = VAR(x_train_)
#model_fit = model.fit()
x = model.select_order(maxlags=12)
x.summary()

# make prediction on validation
#prediction = model_fit.forecast(model_fit.y, steps=len(x_test))


In [ ]:
model_fitted = model.fit(3)


In [ ]:
model_fitted.summary()


In [ ]:
from statsmodels.stats.stattools import durbin_watson
out = durbin_watson(model_fitted.resid)

 #the closer the residual to 2, the less significant the residual
    
for col, val in zip(x_train_rain.columns, out):
    print(col, ':', round(val, 2))
    

In [ ]:
# Forecast Input
x_test_rain_lag = x_test_rain[0:3] 
x_test_rain_lag_noindex = x_test_rain_lag.reset_index(drop=True)
x_train_rain_lag = x_train_rain[-3:].values

In [ ]:
start_date = x_train_rain.index
start_date = list(start_date)[-1]


In [ ]:
fc = model_fitted.forecast(x_train_rain_lag,steps=5)
idx = pd.date_range(start_date, periods=5, freq='T')
df_forecast_rain=pd.DataFrame(data=fc, index=idx, columns=x_test_rain_lag.columns)
df_forecast_rain

In [ ]:
#reset index
df_forecast_rain = df_forecast_rain.reset_index()

In [ ]:
x_test_rain = x_test_rain.reset_index()

In [ ]:
mask1 = x_test_rain["TIMESTAMP"] >= str(df_forecast_rain["index"][0])

In [ ]:
mask2 =  x_test_rain["TIMESTAMP"] <= str(df_forecast_rain["index"][df_forecast_rain.shape[0]-1])

In [ ]:
mask = mask1 & mask2

In [ ]:
x_actual_rain = x_test_rain.loc[mask]